In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data_path = "/content/drive/MyDrive/python/done/wine.csv"

data = pd.read_csv(data_path)
#print(data["Wine"] == 1)
wine1 = data[data["Wine"] == 1]
wine2 = data[data["Wine"] == 2]

plt.scatter(wine1["Color.int"], wine1["Ash"], label="Wine 1")
plt.scatter(wine2["Color.int"], wine2["Ash"], label="Wine 2")
plt.legend()
plt.xlabel("Alcohol")
plt.ylabel("Ash")
plt.xlim([1.0, 9.2])
plt.ylim([1.2, 3.3])
plt.title("Two types of wine")

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input([2]),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss="binary_crossentropy",
    metrics=["accuracy"]
    )



In [ ]:
model.summary()

In [ ]:
import numpy as np

n = len(wine1)+len(wine2)
X = np.zeros((n, 2), dtype = np.float32)
X[:len(wine1), 0] = wine1["Color.int"]
X[:len(wine1), 1] = wine1["Ash"]
X[len(wine1):, 0] = wine2["Color.int"]
X[len(wine1):, 1] = wine2["Ash"]

mean = np.mean(X, axis=0)
stddev = np.std(X, axis=0)
X = (X-mean)/stddev

print(mean)
print(stddev)

y = np.zeros(n, dtype=np.float32)
y[len(wine1):] = 1.0

In [ ]:
model.fit(X, y,batch_size=16, epochs=1000)

In [ ]:
loss, accuracy = model.evaluate(X, y)
print("Accuracy:", accuracy)

In [ ]:
y_pred = model.predict(X)
print(y_pred.shape)
print(y_pred)

In [ ]:
# Color.int, Ash

xx, yy = np.mgrid[0:14:0.1, 0:4:0.01]
grid = (np.c_[xx.ravel(), yy.ravel()]- mean)/stddev

probabilities = model.predict(grid).T.reshape(xx.shape)
plt.contourf(xx, yy, probabilities, 25, cmap="gray", vmin=0.0, vmax=1.0)

plt.scatter(wine1["Color.int"], wine1["Ash"], label="Wine 1")
plt.scatter(wine2["Color.int"], wine2["Ash"], label="Wine 2")
plt.legend()
plt.xlabel("Color")
plt.ylabel("Ash")
plt.xlim([1.0, 9.2])
plt.ylim([1.0, 3.5])
plt.title("Two types of wine")

In [ ]:
n = len(wine1)+len(wine2)
X_data = np.ones((n, 2), dtype = np.float32)
X_data[:len(wine1), 0] = wine1["Color.int"].to_numpy()
X_data[:len(wine1), 1] = wine1["Ash"].to_numpy()
X_data[len(wine1):, 0] = wine2["Color.int"].to_numpy()
X_data[len(wine1):, 1] = wine2["Ash"].to_numpy()

y_data = np.zeros((n, 1), dtype=np.float32)
y_data[len(wine1):] = 1.0

rng = np.random.RandomState(0)
indices = np.arange(len(X_data))
rng.shuffle(indices)

test_fraction = 0.3
test_count = int(len(indices) * test_fraction)
train_indices = indices[test_count:]
test_indices = indices[:test_count]

X_train = X_data[train_indices]
y_train = y_data[train_indices]
X_test = X_data[test_indices]
y_test = y_data[test_indices]

mean = np.mean(X_train, axis=0)
stddev = np.std(X_train, axis=0)

X_train = (X_train - mean) / stddev
X_test = (X_test - mean) / stddev


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="tanh", input_shape=(2,)),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.binary_crossentropy,
    metrics=["accuracy"]
)

model.fit(X_train, y_train, epochs=1000, batch_size=13)

_, accuracy = model.evaluate(X_train, y_train)
print(f"Training accuracy: {accuracy:.4f}")
_, accuracy = model.evaluate(X_test, y_test)
print(f"Testing accuracy: {accuracy:.4f}")

In [ ]:
xx, yy = np.mgrid[0:14:0.1, 0:4:0.01]
grid = (np.c_[xx.ravel(), yy.ravel()] - mean) / stddev

probabilities = model.predict(grid).T.reshape(xx.shape)

In [ ]:
plt.contourf(xx, yy, probabilities, 25, cmap="gray", vmin=0.0, vmax=1.0)
plt.colorbar()

train_wine_table = data.iloc[train_indices]
wine1 = train_wine_table[train_wine_table["Wine"] == 1]
wine2 = train_wine_table[train_wine_table["Wine"] == 2]
plt.scatter(wine1["Color.int"], wine1["Ash"], label="Wine 1", color="C0")
plt.scatter(wine2["Color.int"], wine2["Ash"], label="Wine 2", color="C1")

test_wine_table = data.iloc[test_indices]
wine1 = test_wine_table[test_wine_table["Wine"] == 2]
wine2 = test_wine_table[test_wine_table["Wine"] == 3]
plt.scatter(wine1["Color.int"], wine1["Ash"], color="C0", facecolors="none", lw=2)
plt.scatter(wine2["Color.int"], wine2["Ash"], color="C1", facecolors="none", lw=2)

plt.xlim([1.0, 9.2])
plt.ylim([1.0, 3.5])
plt.xlabel("Color.int")
plt.ylabel("Ash")
plt.legend()

fig = plt.gcf()
fig.set_size_inches(8, 5)